In [5]:
import json
import pandas as pd
import os

In [6]:
pd.set_option('display.max_columns', 30)

In [1]:
#you should be in the main folder with the jupiter notebooks and the srap.py. If not, please change where you are with the os command
os.getcwd()

'c:\\Users\\nicol\\Desktop\\vivino V2'

### This part will collect the data online, clean it, and store it in a csv file, please read the different steps before launching

In [2]:
#Uncomment the code below to lauch the scapper. Warning, it's set for all french red wine, so it will take a while!
#Details on what the crapper does is directly commented in the scrap.py file


#!python scrap.py

In [8]:
# Once all your file are gathered in the resultsscrap folder, we will rename the files for ease of use. 
# Nothing else than the files should be in this folder
i=0
for filename in os.listdir('resultsscrap/'):
    i+=1
    os.rename('resultsscrap/' + filename,'resultsscrap/' +  f"{i}_renamed.json")

In [32]:
# We create a function to merge all the files together
from itertools import chain, starmap


def merge_json_files(file_name, n_files=1):
    """Merges a set of indexed JSON files (1, 2, ..., n) into a new one.
    Args:
        file_name (str): Name of the file to be merged without their indexes and extension.
        n_files (int): Amount of files to be merged.
    Returns:
        The merged file.
    """

    # Creates the data object and its `strains` array
    data = {}
    data['wines'] = []

    # Iterates through every possible file
    for i in range(n_files):
        # Opens the input file
        with open(f'{i+1}_{file_name}', 'r') as f:
            # Loads the temporary data
            tmp_data = json.load(f)

            # Merges the data
            data['wines'] += tmp_data['wines']

        # Closes the file
        f.close()

    # Removes any possible duplicates
    unique_data = list({row['vintage']['seo_name']: row for row in data['wines']}.values())


    

    # Opens the output file
    with open(f'{file_name}', 'w') as f:
        # Dumps the merged data
        json.dump(unique_data, f)



In [10]:
#now moving to the resultsscrap folder. if this is not correct, please fix it or the code below will not work
os.chdir("resultsscrap")
os.getcwd()

'c:\\Users\\nicol\\Desktop\\vivino V2\\resultsscrap'

In [33]:
#we merge all the files using the function written before
merge_json_files("renamed.json", 850)

In [8]:
# We now have one file with our data, we need a function to unpack it as it's a very nested list/dictionnary
from itertools import chain, starmap

def flatten_json_file(json_file):
    """Flattens a multi-level nested JSON file.
    Args:
        json_file (str): Name of the file to be loaded and flattened.
    Returns:
        List containing every flattened record from JSON file.
    """

    def _depack(key, value):
        # Checks whether value is a dictionary
        if isinstance(value, dict):
            # Iterates for every key and value
            for k, v in value.items():
                # Creates a new key string
                new_k = f'{key}_{k}'

                yield new_k, v

        # Checks whether value is a list
        elif isinstance(value, list):
            # Iterates through every value
            for i, v in enumerate(value):
                # Creates a new key string
                new_k = f'{key}_{i}'

                yield new_k, v

        # If the key is not nested
        else:
            yield key, value

    # Opens the JSON file
    with open(json_file, 'r') as f:
        # Loads the JSON file
        json_data = json.load(f)

    # Instantiates a list that will hold every unpacked data
    json_data_list = []

    # Iterates through every sample in the data
    for json_datum in json_data:
        # Performs an all-time true loop
        while True:
            # Unpacks the file
            json_datum = dict(chain.from_iterable(starmap(_depack, json_datum.items())))

            # Creates a loop-break condition
            if not any(isinstance(value, (dict, list)) for value in json_datum.values()):
                break

        # Appends to the list
        json_data_list.append(json_datum)

    return json_data_list

In [11]:
#read and flatten the merged file which is called "renamed"
df= pd.DataFrame(flatten_json_file("renamed.json"))

### Now that we have imported and unpacked the file, we need to clean the data to make it usable
---

In [12]:
#Checking the file content 

# Basic stats
print("Number of rows : {}".format(df.shape[0]))
print("Number of columns : {}".format(df.shape[1]))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)

Number of rows : 18864
Number of columns : 2136

Display of dataset: 


,vintage_id,vintage_seo_name,vintage_name,vintage_statistics_status,vintage_statistics_ratings_count,vintage_statistics_ratings_average,vintage_statistics_labels_count,vintage_image_location,vintage_image_variations_bottle_large,vintage_image_variations_bottle_medium,vintage_image_variations_bottle_medium_square,vintage_image_variations_bottle_small,vintage_image_variations_bottle_small_square,vintage_image_variations_label,vintage_image_variations_label_large,...,vintage_wine_taste_flavor_4_primary_keywords_35_name,vintage_wine_taste_flavor_4_primary_keywords_35_count,vintage_wine_taste_flavor_11_secondary_keywords_18_id,vintage_wine_taste_flavor_11_secondary_keywords_18_name,vintage_wine_taste_flavor_11_secondary_keywords_18_count,vintage_wine_winery,vintage_top_list_rankings_4_rank,vintage_top_list_rankings_4_previous_rank,vintage_top_list_rankings_4_description,vintage_top_list_rankings_4_top_list_id,vintage_top_list_rankings_4_top_list_location,vintage_top_list_rankings_4_top_list_name,vintage_top_list_rankings_4_top_list_seo_name,vintage_top_list_rankings_4_top_list_type,vintage_top_list_rankings_4_top_list_year
0,156171191,chateau-la-gaffeliere-saint-emilion-grand-cru-...,Château La Gaffelière Saint-Émilion Grand Cru ...,Normal,148,4.3,3181,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,//images.vivino.com/thumbs/A_0h7IWDQXOhX8Ud6VX...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2747358,chateau-les-carmes-haut-brion-pessac-leognan-2000,Château les Carmes Haut-Brion Pessac-Léognan 2000,Normal,142,4.3,915,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,//images.vivino.com/thumbs/f5bIiGc6SE-fU5oDYMr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3745068,m-chapoutier-hermitage-monier-de-la-sizeranne-...,M. Chapoutier Hermitage Monier de la Sizeranne...,Normal,139,4.3,888,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,//images.vivino.com/thumbs/SM-AeL7bTge2Ux3zkk-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15441103,chateau-latour-a-pomerol-pomerol-2015,Château Latour à Pomerol Pomerol 2015,Normal,139,4.3,2053,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,//images.vivino.com/thumbs/0f2GktNHSiqfuBa81J-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,156186025,chateau-calon-segur-saint-estephe-grand-cru-cl...,Château Calon-Ségur Saint-Estèphe (Grand Cru C...,Normal,137,4.3,1357,//images.vivino.com/thumbs/fmI0LPDNRKCvTFUurTo...,//images.vivino.com/thumbs/fmI0LPDNRKCvTFUurTo...,//images.vivino.com/thumbs/fmI0LPDNRKCvTFUurTo...,//images.vivino.com/thumbs/fmI0LPDNRKCvTFUurTo...,//images.vivino.com/thumbs/fmI0LPDNRKCvTFUurTo...,//images.vivino.com/thumbs/fmI0LPDNRKCvTFUurTo...,//images.vivino.com/thumbs/fmI0LPDNRKCvTFUurTo...,//images.vivino.com/thumbs/fm


Basics statistics: 


,vintage_id,vintage_seo_name,vintage_name,vintage_statistics_status,vintage_statistics_ratings_count,vintage_statistics_ratings_average,vintage_statistics_labels_count,vintage_image_location,vintage_image_variations_bottle_large,vintage_image_variations_bottle_medium,vintage_image_variations_bottle_medium_square,vintage_image_variations_bottle_small,vintage_image_variations_bottle_small_square,vintage_image_variations_label,vintage_image_variations_label_large,...,vintage_wine_taste_flavor_4_primary_keywords_35_name,vintage_wine_taste_flavor_4_primary_keywords_35_count,vintage_wine_taste_flavor_11_secondary_keywords_18_id,vintage_wine_taste_flavor_11_secondary_keywords_18_name,vintage_wine_taste_flavor_11_secondary_keywords_18_count,vintage_wine_winery,vintage_top_list_rankings_4_rank,vintage_top_list_rankings_4_previous_rank,vintage_top_list_rankings_4_description,vintage_top_list_rankings_4_top_list_id,vintage_top_list_rankings_4_top_list_location,vintage_top_list_rankings_4_top_list_name,vintage_top_list_rankings_4_top_list_seo_name,vintage_top_list_rankings_4_top_list_type,vintage_top_list_rankings_4_top_list_year
count,1.886400e+04,18864,18864,18864,18864.000000,18864.000000,18864.000000,18864,17851,17851,17851,17851,17851,17851,17851,...,1,1.0,1.0,1,1.0,0.0,1.0,1.0,0.0,1.0,1,1,1,1.0,1
unique,NaN,18864,18864,1,NaN,NaN,NaN,7654,6685,6685,6685,6685,6685,6685,6685,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN,1
top,NaN,chateau-la-gaffeliere-saint-emilion-grand-cru-...,Château La Gaffelière Saint-Émilion Grand Cru ...,Normal,NaN,NaN,NaN,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,//images.vivino.com/thumbs/uDMfPG10R1efm5H1do-...,...,viande grillée,NaN,NaN,expresso,NaN,NaN,NaN,NaN,NaN,NaN,_0,Vivino's 2020 Wine Style Awards: Bordeaux Paui...,wine_style_awards_2020_bordeaux_left_bank_paui...,NaN,2020
freq,NaN,1,1,18864,NaN,NaN,NaN,61,61,61,61,61,61,61,61,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN,1
mean,8.690386e+07,NaN,NaN,NaN,243.148219,4.040331,2381.809849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,158.0,NaN,1.0,NaN,3.0,3.0,NaN,2020264.0,NaN,NaN,NaN,6.0,NaN
std,7.373837e+07,NaN,NaN,NaN,432.078757,0.292792,4747.705400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,3.660000e+03,NaN,NaN,NaN,25.000000,2.800000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,158.0,NaN,1.0,NaN,3.0,3.0,NaN,2020264.0,NaN,NaN,NaN,6.0,NaN
25%,3.032805e+06,NaN,NaN,NaN,49.000000,3.800000,379.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,158.0,NaN,1.0,NaN,3.0,3.0,NaN,2020264.0,NaN,NaN,NaN,6.0,NaN
50%,1.418511e+08,NaN,NaN,NaN,102.000000,4.000000,872.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,158.0,NaN,1.0,NaN,3.0,3.0,NaN,2020264.0,NaN,NaN,NaN,6.0,NaN
75%,1.574019e+08,NaN,NaN,NaN,257.000000,4.200000,2536.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,158.0,NaN,1.0,NaN,3.0,3.0,NaN,2020264.0,NaN,NaN,NaN,6.0,NaN


In [47]:
#There are a lot of columns as we have not filtered the data in the scrapper, we will now remove the columns which are not useful to us

listtemp=[]
#here is a list of terms to be removed if they are in the columns name
list_of_columns_to_be_removed=["taste_flavor","image","style","region_country","vintage_top_list_rankings","discount","prices_0",\
    "bottle_type","vintage_grapes","vintage_statistics_ratings",\
    "vintage_wine_taste_structure_fizziness","vintage_wine_id","taste_structure_calculated_structure_count",\
    "taste_structure_user_structure_count","vintage_wine_statistics_labels_count","vintage_wine_statistics_vintages_count",\
    "vintage_wine_has_valid_ratings","vintage_wine_winery_id","vintage_wine_statistics_status",\
    "winery_status","region_id","region_name_en","type_id",\
"price_visibility",'price_sku',"vintage_statistics_labels_count","price_id","vintage_statistics_status",\
"vintage_id","seo_name","price_currency","price_url","vintage_has_valid_ratings"]

for i in df.columns:
    check=0
    for j in list_of_columns_to_be_removed:
        if j in i:
            check+=1
    if check==0: 
        listtemp.append(i)

#also dropping those 3 columns, but their name as part of columns we want to keep so they cannot be in the list before
listtemp = [e for e in listtemp if e not in ["vintage_wine_taste_structure","price","vintage_wine_winery"]]



In [49]:
#checking new dataframe, we should have 15 columns, a much cleaner dataframe
df[listtemp].shape

(18864, 15)

In [50]:
#ckecking for nan
df[listtemp].isna().sum().sort_values(ascending=False)

vintage_wine_taste_structure_acidity       1810
vintage_wine_taste_structure_intensity     1810
vintage_wine_taste_structure_sweetness     1810
vintage_wine_taste_structure_tannin        1810
vintage_wine_winery_name                      1
price_amount                                  1
price_type                                    1
vintage_name                                  0
vintage_wine_name                             0
vintage_wine_vintage_type                     0
vintage_wine_is_natural                       0
vintage_wine_region_name                      0
vintage_wine_statistics_ratings_count         0
vintage_wine_statistics_ratings_average       0
vintage_year                                  0
dtype: int64

In [51]:
#As we have sufficient data, and the data we miss is key, the taste structure, with no way to infer it from other features, 
# we just drop the missing values
df=df[listtemp].dropna()

In [52]:
df.describe(include="all")

,vintage_name,vintage_wine_name,vintage_wine_vintage_type,vintage_wine_is_natural,vintage_wine_region_name,vintage_wine_winery_name,vintage_wine_taste_structure_acidity,vintage_wine_taste_structure_intensity,vintage_wine_taste_structure_sweetness,vintage_wine_taste_structure_tannin,vintage_wine_statistics_ratings_count,vintage_wine_statistics_ratings_average,vintage_year,price_amount,price_type
count,17052,17052,17052.000000,17052,17052,17052,17052.000000,17052.000000,17052.000000,17052.000000,17052.000000,17052.000000,17052.0,17052.000000,17052
unique,17052,3970,NaN,2,421,3140,NaN,NaN,NaN,NaN,NaN,NaN,77.0,NaN,3
top,Château La Gaffelière Saint-Émilion Grand Cru ...,Pomerol,NaN,False,Saint-Émilion Grand Cru,M. Chapoutier,NaN,NaN,NaN,NaN,NaN,NaN,2018.0,NaN,ppc
freq,1,732,NaN,16208,1599,238,NaN,NaN,NaN,NaN,NaN,NaN,2198.0,NaN,10171
mean,NaN,NaN,0.000704,NaN,NaN,NaN,3.818091,3.779541,1.597377,3.417857,6532.880483,4.031891,NaN,249.434540,NaN
std,NaN,NaN,0.030625,NaN,NaN,NaN,0.405751,0.701439,0.199411,0.659413,10016.721825,0.283547,NaN,1866.637053,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN,2.439535,1.000000,1.000000,1.350000,26.000000,3.000000,NaN,3.230000,NaN
25%,NaN,NaN,0.000000,NaN,NaN,NaN,3.435542,3.694888,1.488267,2.991730,742.000000,3.800000,NaN,19.000000,NaN
50%,NaN,NaN,0.000000,NaN,NaN,NaN,3.876403,3.988779,1.597134,3.504724,2148.000000,4.000000,NaN,39.000000,NaN
75%,NaN,NaN,0.000000,NaN,NaN,NaN,4.167666,4.220923,1.700194,3.909170,7736.000000,4.200000,NaN,94.000000,NaN


In [63]:
df["vintage_year"].unique()

array([2018, 2000, 2013, 2015, 1988, 2017, 2008, 2012, 1994, 2016, 2014,
       1986, 1983, 2019, 2020, 2011, 2010, 1967, 2005, 1987, 2006, 2009,
       1982, 1981, 2007, 2002, 2001, 2003, 1978, 1990, 1989, 1979, 1998,
       1984, 1997, 1993, 1975, 1947, 2004, 1996, 1999, 1995, 1985, 1992,
       1980, 1974, 1961, 1950, 1970, 1991, 2021, 1966, 1973, 1977, 2030,
       1952, 1955, 1960, 1968, 1965, 1976, 1972, 1958, 1969, 1959, 1964,
       1971, 1949, 1953, 1945, 1948, 1963, 1957, 1956, 1962, 1946])

In [54]:
#The 8 N.V. (meaning no vintage) are going to cause issue for us, so we will clean it
df["vintage_year"][df["vintage_year"]=="N.V."].value_counts()

N.V.    8
Name: vintage_year, dtype: int64

In [61]:
# a function to replace NV by 2030 and convert the column to integer
def cleanvintage(dataframe):
    dataframe["vintage_year"].replace('N.V.', "2030", inplace=True)
    dataframe["vintage_year"]=dataframe["vintage_year"].astype(int)
    return dataframe

In [62]:
#and we apply it
df= cleanvintage(df)

In [65]:

# understanding what price type means. Data is clean, we keep it
'''
PPC= not available
VC=normal
XDO= price is including a promotion

'''
df["price_type"].value_counts()

ppc    10171
vc      6701
xdo      180
Name: price_type, dtype: int64

In [67]:
#ckecking we cleaned everything and that types are corrects
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17052 entries, 0 to 18863
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   vintage_name                             17052 non-null  object 
 1   vintage_wine_name                        17052 non-null  object 
 2   vintage_wine_vintage_type                17052 non-null  int64  
 3   vintage_wine_is_natural                  17052 non-null  bool   
 4   vintage_wine_region_name                 17052 non-null  object 
 5   vintage_wine_winery_name                 17052 non-null  object 
 6   vintage_wine_taste_structure_acidity     17052 non-null  float64
 7   vintage_wine_taste_structure_intensity   17052 non-null  float64
 8   vintage_wine_taste_structure_sweetness   17052 non-null  float64
 9   vintage_wine_taste_structure_tannin      17052 non-null  float64
 10  vintage_wine_statistics_ratings_count    17052

### Now that we have a clean file, we can save it for future modelling

---

In [69]:
os.chdir("..")
os.getcwd()

'c:\\Users\\nicol\\Desktop\\vivino V2'

In [70]:
#export dataframe to a CSV file
df.to_csv("baseDF.csv")